In [12]:
import numpy as np
import sys
sys.path.append('../Models')
from sklearn.model_selection import train_test_split
from utils import PrimalDualSolver_logistic
from utils import ProjectedGradientDescent_logistic

class SDL_logistic:
    """
    Supervised Dictionary Learning (SDL) Class.

    Combines sparse coding with supervised learning by jointly learning:
    - A dictionary `D` for sparse representation.
    - A linear model `(theta, b)` for predicting labels.
    """

    def __init__(self, n_iter=1000,
                 lamnda0=0.01,
                 lambda1=0.01,
                 lambda2=0.01,
                 lr_D=0.01,
                 lr_theta=0.01,
                 lr_alpha=0.01):
        self.n_iter = n_iter
        self.lamnda0 = lamnda0
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lr_D = lr_D
        self.lr_theta = lr_theta
        self.lr_alpha = lr_alpha

    def objective(self, X, y, D, theta, b, alpha):
        """Computes the objective function value."""
        objective = 0
        for i in range(X.shape[0]):
            xi, yi, ai = X[i], y[i], alpha[i]
            loss_dict = np.linalg.norm(xi - D @ ai)**2
            z = yi * (theta @ ai + b)
            loss_class = np.log(1 + np.exp(-z))
            sparse_penalty = self.lambda1 * np.linalg.norm(ai, 1)
            objective += loss_dict + loss_class + sparse_penalty
        return objective

    def loss_prediction(self, X, y, D, theta, b, alpha):
        """Computes the classification loss."""
        loss = 0
        for i in range(X.shape[0]):
            xi, yi, ai = X[i], y[i], alpha[i]
            z = yi * (theta @ ai + b)
            loss += np.log(1 + np.exp(-z))
        return loss

    def solve_alpha(self, X, y, D, theta, b):
        """
        Optimizes sparse codes `alpha` for fixed `D` and `theta`.
        We can have here a explicit expression of our gradient regarding
        to alpha, so we can use a proximal gradient descent to optimize it.
        """
        n_samples, n_features = X.shape
        alpha = np.zeros((n_samples, n_features))

        for i in range(n_samples):
            x_i = X[i]
            y_i = y[i]

            solver = PrimalDualSolver_logistic(
                theta=theta, b=b, x_i=x_i, y_i=y_i, D=D,
                lambda_0=self.lamnda0, lambda_1=self.lambda1,
                lambd=0.01, mu=1.0
            )
            # Solve the problem
            x0 = np.random.randn(n_features)  # Random initialization
            alpha_opt, _ = solver.solve(x0)

            alpha[i] = alpha_opt

        return alpha

    def solve_D_theta(self, alpha_opt, X, y, D_opt, theta_opt, b):
        """
        Updates `D` and `theta` given the optimal `alpha`.
        Do a projective gradient descent.
        """
        pgd = ProjectedGradientDescent_logistic(
            D_init=D_opt, theta_init=theta_opt,
            b=b, x=X, y=y, alphas=alpha_opt,
            lambda_0=self.lamnda0,
            lambda_1=self.lambda1, lambda_2=self.lambda2,
            lr=self.lr_D, max_iter=self.n_iter
        )
        D_opt, theta_opt, b_opt, _ = pgd.optimize()
        return D_opt, theta_opt, b_opt

    def fit(self, X, y):
        """Fits the model to the data."""
        n_samples, n_features = X.shape
        self.n_components = n_features
        D_opt = np.random.randn(n_features, self.n_components)
        D_opt /= np.linalg.norm(D_opt, axis=0)
        theta_opt = np.zeros(self.n_components)
        b_opt = 0

        for i in range(self.n_iter):
            alpha_opt = self.solve_alpha(X, y, D_opt, theta_opt, b_opt)
            D_opt, theta_opt, b_opt = self.solve_D_theta(alpha_opt,
                                                         X,
                                                         y,
                                                         D_opt,
                                                         theta_opt,
                                                         b_opt)
            # Print the loss of after each iteration
            print(f"Iteration {i+1}/{self.n_iter}, Loss: {self.objective(X, y, D_opt, theta_opt, b_opt, alpha_opt)}")
            print(f"Iteration {i+1}/{self.n_iter}, Loss classification: {self.loss_prediction(X, y, D_opt, theta_opt, b_opt, alpha_opt)}")



        self.alpha = alpha_opt
        self.D = D_opt
        self.theta = theta_opt
        self.b = b_opt

    def predict(self, X):
        """Predicts labels for input data `X`."""
        predictions = []
        for i in range(X.shape[0]):
            x_i = X[i]
            alpha, _, _, _ = np.linalg.lstsq(self.D, x_i, rcond=None)
            prediction = self.theta @ alpha + self.b
            predictions.append(prediction)
        return np.sign(np.array(predictions))

    def score(self, X, y):
        """Computes classification accuracy."""
        y_pred = self.predict(X)
        return np.mean(np.round(y_pred)== y)


In [17]:
# create synthetic data with sin for class 1 and cos for class -1
# do different scaling for the two classes
n_samples = 1000
n_features = 100
n_classes = 2
X = np.random.randn(n_samples, n_features)
y = np.random.choice([-1, 1], n_samples)
X[y == 1] = 2 * np.pi * X[y == 1]
X[y == -1] = 2 * np.pi * X[y == -1] + np.pi
X = np.sin(X)
X = X / np.linalg.norm(X, axis=1)[:, None]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


# Initialize the SDL model
sdl = SDL_logistic(
        n_iter=40,
        lamnda0=0.1,
        lambda1=0.1,
        lambda2=0.001,
        lr_D=0.01,
        lr_theta=0.01,
        lr_alpha=0.01
)


In [18]:
# do a train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Fit the model to the data
sdl.fit(X_train, y_train)

# do the prediction
y_pred_train = sdl.predict(X_train)
print("Train accuracy:", np.mean(np.round(y_pred_train) == y_train))

y_pred_test = sdl.predict(X_test)
print("Test accuracy:", np.mean(np.round(y_pred_test) == y_test))
print("Ground truth labels:", y_test[:10])
print("Predicted labels:", np.round(y_pred_test)[:10])

# count the number of right predictions
print("Number of right predictions:", np.sum(np.round(y_pred_test) == y_test))


Iteration 1/40, Loss: 1269.1370918374034
Iteration 1/40, Loss classification: 469.98125953923267
Iteration 2/40, Loss: 1991.6627763251734
Iteration 2/40, Loss classification: 3.1043462920342457
Iteration 3/40, Loss: 1933.614046497542
Iteration 3/40, Loss classification: 5.947113334778715
Iteration 4/40, Loss: 1901.8518186872316
Iteration 4/40, Loss classification: 6.337183176787359
Iteration 5/40, Loss: 1896.5683182097537
Iteration 5/40, Loss classification: 6.300202271379415
Iteration 6/40, Loss: 1873.8521175368635
Iteration 6/40, Loss classification: 5.732891752703863
Iteration 7/40, Loss: 1890.6294320298841
Iteration 7/40, Loss classification: 5.50324983618307
Iteration 8/40, Loss: 1876.7440234996304
Iteration 8/40, Loss classification: 4.972677652460729
Iteration 9/40, Loss: 1882.4942825991957
Iteration 9/40, Loss classification: 4.6847458487737015
Iteration 10/40, Loss: 1885.1909558048778
Iteration 10/40, Loss classification: 4.569767671912364
Iteration 11/40, Loss: 1871.111570892